In [1]:
from os.path import join as pjoin
import pickle

import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns

In [9]:
site_cols = [f'site{i}' for i in range(1, 11)]
time_cols = [f'time{i}' for i in range(1, 11)]
name_cols = [f'name{i}' for i in range(1, 11)]

In [10]:
train_df = pd.read_csv(pjoin('data', 'train_sessions.csv'),
                       index_col='session_id',
                       parse_dates=time_cols,
                      )
test_df = pd.read_csv(pjoin('data', 'test_sessions.csv'),
                      index_col='session_id',
                      parse_dates=time_cols,
                     )
df = pd.concat([train_df, test_df],
               ignore_index=True)
with open(pjoin('data', 'site_dic.pkl'), 'rb') as file:
    site_names = pickle.load(file)
site_names = {w: k for k, w in site_names.items()}
site_names[-1] = ''
df.head()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time1,time10,time2,time3,time4,time5,time6,time7,time8,time9
0,718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-02-20 10:02:45,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,2014-02-22 11:19:50,2014-02-22 11:20:16,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15
2,14769,14768.0,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,...,2013-12-16 16:40:17,2013-12-16 16:40:24,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22
3,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,...,2014-03-28 10:52:12,2014-03-28 10:56:42,2014-03-28 10:52:42,2014-03-28 10:53:12,2014-03-28 10:53:42,2014-03-28 10:54:12,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12
4,22,178.0,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,...,2014-02-28 10:53:05,2014-02-28 10:57:11,2014-02-28 10:55:22,2014-02-28 10:55:22,2014-02-28 10:55:23,2014-02-28 10:55:23,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06


In [11]:
df[site_cols] = df[site_cols].fillna(-1).astype('int32')
df['target'] = df['target'].fillna(-1).astype('int8')
df['session_len'] = (df[site_cols] > 0).sum(1).astype('uint8')

In [12]:
df[name_cols] = df[site_cols].applymap(site_names.get)
df = df.drop(site_cols, 1)

In [30]:
df['hour'] = df['time1'].map(lambda x: x.hour).astype('uint8')
df['week'] = df['time1'].map(lambda x: x.weekday()).astype('uint8')
df['month'] = df['time1'].map(lambda x: x.month).astype('uint8')
df['time_int'] = (df[time_cols].max(1) - df['time1']).map(lambda x: x.seconds).astype('uint16')
df = df.drop(time_cols, 1)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336358 entries, 0 to 336357
Data columns (total 16 columns):
target         336358 non-null int8
session_len    336358 non-null uint8
name1          336358 non-null object
name2          336358 non-null object
name3          336358 non-null object
name4          336358 non-null object
name5          336358 non-null object
name6          336358 non-null object
name7          336358 non-null object
name8          336358 non-null object
name9          336358 non-null object
name10         336358 non-null object
hour           336358 non-null uint8
week           336358 non-null uint8
month          336358 non-null uint8
time_int       336358 non-null uint16
dtypes: int8(1), object(10), uint16(1), uint8(4)
memory usage: 27.9+ MB


In [42]:
train_df = df[df['target'].notna()]
test_df = df[df['target'].isna()]
del test_df['target']